In [1]:
from writers import foyer_xml_writer
from writers.foyer_xml_writer import parmed_to_foyer_xml, mbuild_to_foyer_xml


import ele
import espaloma as esp
import forcefield_utilities as ffutils
import foyer
import gmso
import mbuild as mb
from mbuild.lib.recipes import Polymer
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import numpy as np
from openff.toolkit.topology import Molecule
import torch


import os
import warnings
warnings.filterwarnings("ignore")

if not os.path.exists("espaloma_model.pt"):
    os.system("wget http://data.wangyq.net/espaloma_model.pt")

/Users/madilyn/miniconda3/envs/esp/lib/python3.9/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as soon as v0.13.0
  warn(
/Users/madilyn/miniconda3/envs/esp/lib/python3.9/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as soon as v0.13.0
  warn(
LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


In [2]:
# Use espaloma to find the FF parameters for Benzene
smiles = "c1ccccc1"
benzene = mb.load(smiles, smiles=True)
#benzene.visualize().show()

molecule = Molecule.from_smiles(smiles) #molecule is benzene
molecule_graph = esp.Graph(molecule)

espaloma_model = torch.load("espaloma_model.pt")
espaloma_model(molecule_graph.heterograph)
openmm_system = esp.graphs.deploy.openmm_system_from_graph(molecule_graph)

# Store the results for each in something more accessible
pair_forces = openmm_system.getForces()[1]
angle_forces = openmm_system.getForces()[3]
bond_forces = openmm_system.getForces()[2]
torsion_forces = openmm_system.getForces()[0]

In [ ]:
print(pair_forces.getParticleParameters(0)) 
#some issues with the data that comes out of pair_forces.getParticleParameters:
    # WE have  list of "QUantities", which is nice, but we only want the values (but need to know units)
    # WE need informatio nabout which particle type each of these quanitities corresponds to
help(openmm_system.getForces)

In [ ]:
#structure.bonds

In [3]:
# get a parmed structure from openmm 
import parmed as pmd
topology = molecule.to_topology()
openmm_topology = topology.to_openmm()

structure = pmd.openmm.load_topology(topology=openmm_topology, system=openmm_system)
structure.bonds.sort(key=lambda x: x.atom1.idx)

In [4]:
# step 1: how to get bond parameters

bond_parameters = {}

for bond in structure.bonds:
    bond_parameters[(bond.atom1.name, bond.atom2.name)] = {"k":[bond.type.k] , "l0":[bond.type.req]}
    
# step 2: get angle parameters

angle_parameters = {}

for angle in structure.angles:
    angle_parameters[(angle.atom1.name, angle.atom2.name,angle.atom3.name)] = {"k":[angle.type.k], 
                                                                               "t0":[angle.type.theteq]}
    
# step 3: dihedral parameters

dihedral_parameters = {}

for dihedral in structure.dihedrals:
    dihedral_parameters[(dihedral.atom1.name, dihedral.atom2.name,dihedral.atom3.name,
                         dihedral.atom4.name)] = {"periodicity":[dihedral.type.per],
                                                  "k":[dihedral.type.phi_k],"phase":[dihedral.type.phase]}
    
# step 4: non-bonding parameters

nonbonded_parameters = {}

for nonbonded in structure.adjusts:
    nonbonded_parameters[(nonbonded.atom1.name,nonbonded.atom2.name)] = {"sigma":[nonbonded.type.rmin],
                                                    "epsilon":[nonbonded.type.epsilon],
                                                    "charge":[nonbonded.type.chgscale]}

In [ ]:
#bond_parameters
#angle_parameters
#dihedral_parameters
#nonbonded_parameters

In [ ]:
mbuild_to_foyer_xml(
    file_name="benzene_esp.xml",
    compound=benzene,
    bond_params=bond_parameters,
    angle_params=angle_parameters,
    dihedral_params=dihedral_parameters,
    dihedral_type="periodic",
    non_bonded_params=nonbonded_parameters,
    combining_rule="geometric",
    name="",
    version="",
    coulomb14scale=1.0,
    lj14scale=1.0)

In [ ]:
##creating espaloma xml without parmed

In [5]:
particle_types = []
particle_type_dict = dict()

for i in range(pair_forces.getNumParticles()):
    pair_parms = pair_forces.getParticleParameters(index=i)
    charge = pair_parms[0]
    sigma = pair_parms[1]
    epsilon = pair_parms[2]
    if (charge, sigma, epsilon) not in particle_types: #this is adding the charge,sigma, and epsilon to the particle_types, but only once.
        particle_types.append((charge, sigma, epsilon))
        #print((charge, sigma, epsilon) )
    particle_type_dict[i] = particle_types.index((charge, sigma, epsilon)) #adding only the index of the pair_parms, this gives particle types.
print(particle_type_dict)

#particle_types gives the sigma, charge, and epsilon associated with each unique particle, 
#particle_type_dict only gives the index associated with each unique particle type

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1}


In [6]:
#graph check
#want to check if the opemm and parmed graphs are the same so we can use types from parmed 
import networkx  as nx
Gopenmm = nx.Graph()
Gparmed = nx.Graph()
#openmm:
for i in range(bond_forces.getNumBonds()):
    Gopenmm.add_edge(bond_forces.getBondParameters(index=i)[0],bond_forces.getBondParameters(index=i)[1])
#parmed
for b in structure.bonds:
    Gparmed.add_edge(b.atom1.idx,b.atom2.idx)
type_map = {}
#nx.rooted_tree_isomorphism
#in here we still need to check that one known index on one corresponds to the same index on the other....
tree_openmm = nx.bfs_tree(Gopenmm,0)
tree_parmed = nx.bfs_tree(Gparmed,0)
if nx.is_isomorphic(Gopenmm,Gparmed):
#if nx.isomorphism.tree_isomorphism(tree_openmm,tree_parmed):  <- want this work
    for b in structure.bonds:
        type_map[b.atom1.idx] = b.atom1.atom_type.__str__()
        type_map[b.atom2.idx] = b.atom2.atom_type.__str__()
print(type_map)

{0: 'C1', 1: 'C1', 6: 'H1', 2: 'C1', 7: 'H1', 3: 'C1', 8: 'H1', 4: 'C1', 9: 'H1', 5: 'C1', 10: 'H1', 11: 'H1'}


In [ ]:
nx.is_tree(Gopenmm)#,Gopenmm.nodes[0],Gparmed,Gparmed.nodes[0])

In [ ]:
#structure.bonds[1].atom1.atom_type.__str__()

In [7]:
bond_types = []
bond_dict = dict() #creating an empty dictionary to put our parameters in 

for i in range(bond_forces.getNumBonds()):
    bond_parms = bond_forces.getBondParameters(index=i)
    #print(bond_parms)
    #atoms = {(bond.atom1.name,bond.atom2.name):[]} #gives you the atom type name and index
    l0 = bond_parms[2]/bond_parms[2].unit
    k = bond_parms[3]/bond_parms[3].unit
    #if (type_map[bond_parms[0]],type_map[bond_parms[1]],k,l0) not in bond_dict:
    #    bond_types.append((type_map[bond_parms[0]],type_map[bond_parms[1]],k,l0))
    bond_dict[type_map[bond_parms[0]],type_map[bond_parms[1]]] = {'k':[k],'l0':[l0]}
print(bond_dict)

{('C1', 'C1'): {'k': [130064.45891205409], 'l0': [0.14419899337180256]}, ('C1', 'H1'): {'k': [123800.77912066571], 'l0': [0.11234977990290071]}}


In [8]:
angle_types = []
angle_dict = dict() #creating an empty dictionary to put our parameters in 

for i in range(angle_forces.getNumAngles()):
    angle_parms = angle_forces.getAngleParameters(index=i)
    k = angle_parms[4]/angle_parms[4].unit
    t0 = angle_parms[3]/angle_parms[3].unit  
#    if (type_map[angle_parms[0]],type_map[angle_parms[1]],type_map[angle_parms[2]],k,t0) not in angle_types:
#        angle_types.append((type_map[angle_parms[0]],type_map[angle_parms[1]],type_map[angle_parms[2]],k,t0))
    angle_dict[type_map[angle_parms[0]],type_map[angle_parms[1]],type_map[angle_parms[2]]] = {'k':[k],'t0':[t0]}
print(angle_dict)

{('C1', 'C1', 'C1'): {'k': [472.10858457532197], 't0': [2.1592049598693848]}, ('C1', 'C1', 'H1'): {'k': [492.405515275309], 't0': [2.083374500274658]}}


In [9]:
dihedral_types = []
dihedral_dict = {}

for i in range(torsion_forces.getNumTorsions()):
    dihedral_parms = torsion_forces.getTorsionParameters(index=i)
    periodicity = dihedral_parms[4]
    phase = dihedral_parms[5]/dihedral_parms[5].unit
    k = dihedral_parms[6]/dihedral_parms[6].unit
#    if (type_map[dihedral_parms[0]],type_map[dihedral_parms[1]],type_map[dihedral_parms[2]],type_map[dihedral_parms[3]],periodicity,k,phase) not in dihedral_types:
#        dihedral_types.append((type_map[dihedral_parms[0]],type_map[dihedral_parms[1]],type_map[dihedral_parms[2]],
#        type_map[dihedral_parms[3]],periodicity,k,phase))
    dihedral_dict[type_map[dihedral_parms[0]],type_map[dihedral_parms[1]],type_map[dihedral_parms[2]],
                  type_map[dihedral_parms[3]]] = {'periodicity':[periodicity],'k':[k],'phase':[phase]}
print(dihedral_dict)

{('C1', 'C1', 'C1', 'C1'): {'periodicity': [6], 'k': [0.09353574152013451], 'phase': [0.0]}, ('C1', 'C1', 'C1', 'H1'): {'periodicity': [6], 'k': [1.1611704423988047], 'phase': [0.0]}, ('H1', 'C1', 'C1', 'H1'): {'periodicity': [6], 'k': [0.3411427211963267], 'phase': [3.141592653589793]}, ('C1', 'C1', 'H1', 'C1'): {'periodicity': [6], 'k': [1.1611704423988047], 'phase': [0.0]}, ('H1', 'C1', 'C1', 'C1'): {'periodicity': [6], 'k': [4.569218405812133], 'phase': [3.141592653589793]}}


In [13]:
nonbonded_types = []
nonbonded_dict = {}

for i in range(pair_forces.getNumParticles()):
    nonbonded_parms = pair_forces.getParticleParameters(index=i)
    charge = nonbonded_parms[0]/nonbonded_parms[0].unit
    sigma = nonbonded_parms[1]/nonbonded_parms[1].unit
    epsilon = nonbonded_parms[2]/nonbonded_parms[2].unit
    #atom_type = structure.adjusts.atom
    if (charge,sigma,epsilon) not in nonbonded_types:
        nonbonded_types.append((charge,sigma,epsilon))
    nonbonded_dict[(type_map[i])]={'charge':[charge],'sigma':[sigma],'epsilon':[epsilon]}
print(nonbonded_dict)

{'C1': {'charge': [-0.13], 'sigma': [0.3399669508423535], 'epsilon': [0.359824]}, 'H1': {'charge': [0.13], 'sigma': [0.25996424595335105], 'epsilon': [0.06276]}}


In [14]:
mbuild_to_foyer_xml(
    file_name="benzene_esp_nonparmed.xml",
    compound=benzene,
    bond_params=bond_dict,
    angle_params=angle_dict,
    dihedral_params=dihedral_dict,
    dihedral_type="periodic",
    non_bonded_params=nonbonded_dict,
    combining_rule="geometric",
    name="",
    version="",
    coulomb14scale=1.0,
    lj14scale=1.0)

In [ ]:
#nonbonded_types = []
#nonbonded_dict = {}
#
#for a in structure.atoms:
#    i = a.idx
#    t = a.atom_type.__str__()
#    nonbonded_parms = pair_forces.getParticleParameters(index=i)
#    charge = nonbonded_parms[0]/nonbonded_parms[0].unit
#    sigma = nonbonded_parms[1]/nonbonded_parms[1].unit
#    epsilon = nonbonded_parms[2]/nonbonded_parms[2].unit
#    print(charge,sigma,epsilon)
#    if (t,t) not in nonbonded_dict:
#        nonbonded_dict[(t,t)] = {"charge":[charge],"sigma":[sigma],"epsilon":[epsilon]}
#print(nonbonded_dict)

In [ ]:
#Checking indicies are in same order between parmed and openmm:
    # bond graphs are same - necessary
    # index 0 same on both graphs 
    
#Describing graph is with edges: (our bond list!)
#C1x-H1x
#C1x-C2x
#...

#Numbered graph edges
#0: 0-1
#1: 1-2
#3: 2-3

In [ ]:
#nonbonded_dict = {}
#
#for i in range(pair_forces.getNumParticles()):
#nonbonded_parms = pair_forces.getParticleParameters(index=i)
#charge = nonbonded_parms[0]/nonbonded_parms[0].unit
#sigma = nonbonded_parms[1]/nonbonded_parms[1].unit
#epsilon = nonbonded_parms[2]/nonbonded_parms[2].unit
#print(i,charge,sigma,epsilon)
#print(nonbonded_dict)

In [ ]:
##using parmed to get atom types
#
#atom1_names = {}
#for bond in structure.bonds:
#    atom1_names = {"atom1":[bond.atom1.name,bond.atom1.idx],"atom2":[bond.atom2.name, bond.atom2.idx]} #gives you the atom type name and index
##    print(atom1_names)
#atoms = {}
#for bond in structure.bonds:
#    atoms = {(bond.atom1.name,bond.atom2.name):[bond.atom1.idx,bond.atom2.idx]} #gives you the atom type name and index
#    print(atoms)

In [ ]:
#dihedral_parameters